# Загрузка данных

In [ ]:
import os
import cv2
from bs4 import BeautifulSoup

# Определите пути к папкам с изображениями и аннотациями
images_path = 'stanford dogs dataset/images/Images'
annotations_path = 'stanford dogs dataset/annotations/Annotation'

# Функция для чтения аннотаций
def read_annotation(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        bs_content = BeautifulSoup(content, 'lxml')
        return bs_content.find('name').get_text()

# Функция для чтения изображений
def read_image(file_path):
    return cv2.imread(file_path)

# Функция для получения списка всех путей к изображениям и соответствующих им меток классов
def get_dataset(images_path, annotations_path):
    images = []
    labels = []
    for breed_folder in os.listdir(images_path):
        breed_images_path = os.path.join(images_path, breed_folder)
        breed_annotations_path = os.path.join(annotations_path, breed_folder)
        if os.path.isdir(breed_images_path):  # Check if it's a directory
            for image_file in os.listdir(breed_images_path):
                image_path = os.path.join(breed_images_path, image_file)
                annotation_file = image_file.split('.')[0]
                annotation_path = os.path.join(breed_annotations_path, annotation_file)
                if os.path.isfile(annotation_path):  # Check if it's a file
                    images.append(read_image(image_path))
                    labels.append(read_annotation(annotation_path))
    return images, labels
# Создайте набор данных
images, labels = get_dataset(images_path, annotations_path)

# Предварительная обработка данных

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer

def process_image(image):
    image = cv2.resize(image, (224, 224))
    image = np.uint8(image / 255.0 * 255)  # Convert image back to 8-bit
    return image

def one_hot_encode_labels(labels):
    lb = LabelBinarizer()
    return lb.fit_transform(labels)

labels = one_hot_encode_labels(labels)

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = [executor.submit(process_image, image) for image in images]
    images = []
    for future in tqdm(as_completed(futures), total=len(futures)):
        images.append(future.result())

images = np.array(images)
images = images / 255.0  # Нормализация изображений

labels = np.array(labels)

# Создание модели

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, BatchNormalization, Dropout

# Создание модели
model = Sequential()

# Добавление слоя Input
model.add(Input(shape=(224, 224, 3)))

# Добавление первого сверточного слоя, слоя Batch Normalization, слоя Dropout и слоя пулинга
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))  # Add Dropout layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Добавление второго сверточного слоя, слоя Batch Normalization, слоя Dropout и слоя пулинга
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))  # Add Dropout layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Добавление третьего сверточного слоя, слоя Batch Normalization, слоя Dropout и слоя пулинга
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))  # Add Dropout layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Выравнивание входа
model.add(Flatten())

# Добавление полносвязного слоя и слоя Dropout
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add Dropout layer

# Добавление выходного слоя
model.add(Dense(120, activation='softmax'))

# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели

In [ ]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import numpy as np

# Разделите данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Создайте callback для ранней остановки
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Обучите модель
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Оценка модели

In [ ]:
# Оцените модель на тестовом наборе данных
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print(f"Test accuracy: {test_accuracy}")
print(f"Test loss: {test_loss}")